# SNEWS_PT Examples


## Publication

In [1]:
from snews_pt.snews_pub import SNEWSTiersPublisher
from datetime import  datetime

test_time = datetime.utcnow().isoformat()
message = SNEWSTiersPublisher(detector_name='LZ',machine_time=test_time ,neutrino_time=test_time, p_val=0.0007, p_values=[0.0007,0.0008,0.0009],t_bin_width=0.07)
print(message.message_data)
print(message.tiernames)
print('Sending message')
message.send_to_snews()

{'detector_name': 'LZ', 'machine_time': '2022-08-05T15:49:36.711267', 'neutrino_time': '2022-08-05T15:49:36.711267', 'p_val': 0.0007, 'p_values': [0.0007, 0.0008, 0.0009], 't_bin_width': 0.07, 'timing_series': None, 'which_tier': None, 'n_retract_latest': None, 'retraction_reason': None, 'detector_status': None, 'is_pre_sn': False}
['CoincidenceTier', 'SigTier']
Sending message
----------------------------------------------------------------
Sending message to CoincidenceTier on kafka://kafka.scimma.org/snews.experiments-firedrill
_id                 :14_CoincidenceTier_2022-08-05T15:49:36.711267
detector_name       :LZ
machine_time        :2022-08-05T15:49:36.711267
neutrino_time       :2022-08-05T15:49:36.711267
p_val               :0.0007
meta                :{}
schema_version      :1.1.0
sent_time           :2022-08-05T15:49:36.711322
----------------------------------------------------------------
Sending message to SigTier on kafka://kafka.scimma.org/snews.experiments-firedrill


/Users/joe/src/gitjoe/SNEWS_Publishing_Tools/snews_pt/__init__.py:19: UserWarning: You are using default detector name "TEST"
Please change this by snews_pt.snews_pt_utils.set_name()
  warnings.warn(warning_text, UserWarning)


_id                 :14_SigTier_2022-08-05T15:49:36.711267
detector_name       :LZ
machine_time        :2022-08-05T15:49:36.711267
neutrino_time       :2022-08-05T15:49:36.711267
p_values            :[0.0007, 0.0008, 0.0009]
t_bin_width         :0.07
meta                :{}
schema_version      :1.1.0
sent_time           :2022-08-05T15:49:36.711322


## Subscription

In [2]:
from snews_pt.snews_sub import Subscriber

Subscriber().subscribe()

You are subscribing to ALERT
Broker:kafka://kafka.scimma.org/snews.alert-firedrill
Done
